<a href="https://colab.research.google.com/github/RajeshworM/IMPDS_Datafrom_Portal/blob/main/IMPDS_India.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size ='6'> <font color ='green'>  **Portal data fething program for All India - IMPDS Data**

In [1]:
import pandas as pd
import requests
import time
from io import StringIO
import warnings
warnings.filterwarnings('ignore')

# Configuration
start_year = 2020  # You can change this as needed
end_year = 2025
end_month_2025 = 11

print("Starting All India data extraction for 4 table types...")
print("=" * 70)

# Store data for all 4 types
type1_data = []  # Transaction Counts (Table 3)
type2_data = []  # Card/Beneficiary Stats (Table 2)
type3_data = []  # Transacted Ration Cards (Table 4)
type4_data = []  # Distributed Quantity (Table 5)

def extract_all_four_tables(url, month, year):
    """Extract all 4 table types based on their exact structure"""
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()

        # Get all tables
        tables = pd.read_html(StringIO(response.text))

        found_types = []

        # Table 2: Type 2 - Card/Beneficiary Statistics (multi-index)
        if len(tables) > 2:
            table2 = tables[2]
            if isinstance(table2.columns, pd.MultiIndex):
                print(f"    ✓ Table 2: Type 2 (Card/Beneficiary Stats)")
                df = table2.copy()

                # Flatten multi-index columns
                df.columns = ['_'.join(filter(None, map(str, col))).strip()
                            for col in df.columns.values]

                df.columns = [str(col).strip() for col in df.columns]
                df['Month'] = month
                df['Year'] = year
                df['Data_Type'] = 'Card_Beneficiary_Stats'
                type2_data.append(df)
                found_types.append('Type2')

        # Table 3: Type 1 - Transaction Counts
        if len(tables) > 3:
            table3 = tables[3]
            cols_lower = ' '.join([str(col).lower() for col in table3.columns])
            if 'nfsa ration card type' in cols_lower:
                print(f"    ✓ Table 3: Type 1 (Transaction Counts)")
                df = table3.copy()
                df.columns = [str(col).strip() for col in df.columns]
                df['Month'] = month
                df['Year'] = year
                df['Data_Type'] = 'Transaction_Counts'
                type1_data.append(df)
                found_types.append('Type1')

        # Table 4: Type 3 - Transacted Ration Cards
        if len(tables) > 4:
            table4 = tables[4]
            cols_lower = ' '.join([str(col).lower() for col in table4.columns])
            if 'nfsa ration card type' in cols_lower:
                print(f"    ✓ Table 4: Type 3 (Transacted Ration Cards)")
                df = table4.copy()
                df.columns = [str(col).strip() for col in df.columns]
                df['Month'] = month
                df['Year'] = year
                df['Data_Type'] = 'Transacted_Ration_Cards'
                type3_data.append(df)
                found_types.append('Type3')

        # Table 5: Type 4 - Distributed Quantity
        if len(tables) > 5:
            table5 = tables[5]
            cols_lower = ' '.join([str(col).lower() for col in table5.columns])
            if 'commodity' in cols_lower:
                print(f"    ✓ Table 5: Type 4 (Distributed Quantity)")
                df = table5.copy()
                df.columns = [str(col).strip() for col in df.columns]
                df['Month'] = month
                df['Year'] = year
                df['Data_Type'] = 'Distributed_Quantity'
                type4_data.append(df)
                found_types.append('Type4')

        return len(found_types)

    except Exception as e:
        print(f"  ✗ Error: {str(e)[:50]}")
        return 0

# Main extraction loop
for year in range(start_year, end_year + 1):
    for month in range(1, 13):
        if year == 2025 and month > end_month_2025:
            break

        url = f"https://impds.nic.in/sale/stateUnautmated?month={month}&year={year}"

        print(f"\nProcessing {month:02d}/{year}:")

        found = extract_all_four_tables(url, month, year)

        if found == 4:
            print(f"  ✓ All 4 types found")
        elif found > 0:
            print(f"  ⚠ Found {found}/4 types")
        else:
            print(f"  ✗ No types found")

        time.sleep(1)  # Be polite to the server

print("\n" + "="*70)
print("EXTRACTION COMPLETE!")
print("="*70)

print(f"\nSUMMARY:")
print(f"Type 1 (Transaction Counts): {len(type1_data)} months")
print(f"Type 2 (Card/Beneficiary): {len(type2_data)} months")
print(f"Type 3 (Transacted Cards): {len(type3_data)} months")
print(f"Type 4 (Distributed Quantity): {len(type4_data)} months")

print("\n" + "="*70)
print("SAVING FILES...")
print("="*70)

# Save all files
files_saved = []

# Type 1
if type1_data:
    df1 = pd.concat(type1_data, ignore_index=True)
    df1 = df1.drop_duplicates()
    filename1 = f"All_India_Transaction_Counts_{start_year}_{end_year}.csv"
    df1.to_csv(filename1, index=False, encoding='utf-8-sig')
    files_saved.append(filename1)
    print(f"\n✓ Type 1: {filename1}")
    print(f"  Rows: {len(df1)}, Columns: {len(df1.columns)}")

# Type 2
if type2_data:
    df2 = pd.concat(type2_data, ignore_index=True)
    df2 = df2.drop_duplicates()
    filename2 = f"All_India_Card_Beneficiary_Stats_{start_year}_{end_year}.csv"
    df2.to_csv(filename2, index=False, encoding='utf-8-sig')
    files_saved.append(filename2)
    print(f"\n✓ Type 2: {filename2}")
    print(f"  Rows: {len(df2)}, Columns: {len(df2.columns)}")

# Type 3
if type3_data:
    df3 = pd.concat(type3_data, ignore_index=True)
    df3 = df3.drop_duplicates()
    filename3 = f"All_India_Transacted_Ration_Cards_{start_year}_{end_year}.csv"
    df3.to_csv(filename3, index=False, encoding='utf-8-sig')
    files_saved.append(filename3)
    print(f"\n✓ Type 3: {filename3}")
    print(f"  Rows: {len(df3)}, Columns: {len(df3.columns)}")

# Type 4
if type4_data:
    df4 = pd.concat(type4_data, ignore_index=True)
    df4 = df4.drop_duplicates()
    filename4 = f"All_India_Distributed_Quantity_MT_{start_year}_{end_year}.csv"
    df4.to_csv(filename4, index=False, encoding='utf-8-sig')
    files_saved.append(filename4)
    print(f"\n✓ Type 4: {filename4}")
    print(f"  Rows: {len(df4)}, Columns: {len(df4.columns)}")

print("\n" + "="*70)
print("DOWNLOADING FILES...")
print("="*70)

# Download files
try:
    from google.colab import files

    for filename in files_saved:
        print(f"\nDownloading {filename}...")
        files.download(filename)
        print(f"✓ Downloaded")

    print(f"\n✓ Downloaded {len(files_saved)} files")

except ImportError:
    print("\nRunning locally. Files saved:")
    for filename in files_saved:
        print(f"  • {filename}")

print("\n" + "="*70)
print("ALL DONE!")
print("="*70)

Starting All India data extraction for 4 table types...

Processing 01/2020:
    ✓ Table 2: Type 2 (Card/Beneficiary Stats)
    ✓ Table 3: Type 1 (Transaction Counts)
    ✓ Table 4: Type 3 (Transacted Ration Cards)
    ✓ Table 5: Type 4 (Distributed Quantity)
  ✓ All 4 types found

Processing 02/2020:
    ✓ Table 2: Type 2 (Card/Beneficiary Stats)
    ✓ Table 3: Type 1 (Transaction Counts)
    ✓ Table 4: Type 3 (Transacted Ration Cards)
    ✓ Table 5: Type 4 (Distributed Quantity)
  ✓ All 4 types found

Processing 03/2020:
    ✓ Table 2: Type 2 (Card/Beneficiary Stats)
    ✓ Table 3: Type 1 (Transaction Counts)
    ✓ Table 4: Type 3 (Transacted Ration Cards)
    ✓ Table 5: Type 4 (Distributed Quantity)
  ✓ All 4 types found

Processing 04/2020:
    ✓ Table 2: Type 2 (Card/Beneficiary Stats)
    ✓ Table 3: Type 1 (Transaction Counts)
    ✓ Table 4: Type 3 (Transacted Ration Cards)
    ✓ Table 5: Type 4 (Distributed Quantity)
  ✓ All 4 types found

Processing 05/2020:
    ✓ Table 2: Type

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ Downloaded



<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

✓ Downloaded



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ Downloaded

✓ Downloaded 4 files

ALL DONE!
